In [6]:
#lilo
%cd '/content/drive/MyDrive/Colab Notebooks/preferential-attachment/notebooks'

/content/drive/MyDrive/Colab Notebooks/preferential-attachment/notebooks


In [11]:
# !git config user.email "yossi.cohen@live.com"
# !git config user.name "Yossi Cohen"
# !git commit -am "poisson-exp-3" && git push